# Babamul streaming example

In this notebook we'll read from Babamul streams to fetch alerts of interest
and display them interactively.

🚨 **Before running, be sure to copy `.env.example` to `.env` and fill in
your Babamul Kafka credentials from https://babamul.caltech.edu/profile.** 🚨

In [ ]:
import dotenv
from astropy.coordinates import SkyCoord
from astropy.table import Table
from tqdm.notebook import tqdm

import babamul
from babamul import LsstAlert, ZtfAlert

In [ ]:
dotenv.load_dotenv()

In [ ]:
# Define some parameters
limit = 1000
topics = ["babamul.lsst.ztf-match.hosted"]

In [ ]:
alerts = []

for alert in babamul.AlertConsumer(
    topics=topics,
    offset="earliest",
    timeout=2,
):
    alerts.append(alert)
    if len(alerts) >= limit:
        break

print(f"Fetched {len(alerts)} alerts.")

In [ ]:
alert = alerts[7]
print(f"Showing {alert.objectId} with drb={alert.drb}:")
alert.show()

In [ ]:
def is_relevant(alert: ZtfAlert | LsstAlert):
    if isinstance(alert, ZtfAlert):
        age = alert.candidate.jd - alert.candidate.jdstarthist
        # Only consider alerts for objects that were observed for the first time within the last 30 days
        if age > 60:
            return False
    if isinstance(alert, LsstAlert):
        if alert.candidate.isDipole:
            return False
        if alert.candidate.psfFlux_flag:
            return False

    # Only consider alerts with a real-bogus score above 0.2
    if alert.drb < 0.2:
        return False

    # Exclude alerts that are likely to be known SSOs
    if alert.properties.rock:
        return False

    # Exclude alerts that are likely to be stars or near bright stars
    if alert.properties.star:  # using PS1 PSC for ZTF, using LSPSC for LSST
        return False

    if alert.properties.near_brightstar:  # same here
        return False

    # Only consider positive subtractions (i.e. candidate is brighter than the reference)
    if not alert.candidate.isdiffpos:
        return False

    # Exclude alerts that are not "stationary", i.e. not detected at least twice with sufficient time separation
    # (helps rule out uncataloged asteroids, some boguses, ...)
    if not alert.properties.stationary:
        return False

    # Exclude alerts that match with knowns QSOs (from milliquas v8)
    cross_matches: babamul.models.CrossMatches = alert.get_cross_matches()
    return not (
        cross_matches
        and cross_matches.milliquasar
        and len(cross_matches.milliquasar) > 0
    )

In [ ]:
limit = 20
alerts: list[LsstAlert] = []
for alert in tqdm(
    babamul.AlertConsumer(
        topics=topics,
        offset="earliest",
        timeout=2,
    ),
    desc="Filtering alerts",
):
    if is_relevant(alert):
        alerts.append(alert)
    cross_matches = alert.get_cross_matches()
    if not cross_matches or not cross_matches.ned:
        continue
    if len(alerts) >= limit:
        break

print(f"Fetched {len(alerts)} alerts.")

In [ ]:
alert = alerts[5]
alert.show()

In [ ]:
# We deduplicate by objectId, keeping the most recent alert for each
# scan for these alerts
alerts_to_scan = {}
for a in alerts:
    if (
        a.objectId not in alerts_to_scan
        or a.candidate.jd > alerts_to_scan[a.objectId].candidate.jd
    ):
        alerts_to_scan[a.objectId] = a
alerts_to_scan = list(alerts_to_scan.values())

## View alerts interactively

The cell below allows us to page through the alerts we have selected and view their contents interactively.

In [ ]:
babamul.scan_alerts(alerts_to_scan)

In [ ]:
def has_nearby_host(alert: ZtfAlert | LsstAlert):
    # we use the API to retrieve the cross-matches for this alert's object
    cross_matches = alert.get_cross_matches()
    # check if we have at least one match w/ NED
    if not cross_matches.ned:
        return False
    # at least one of them should be at z <= 0.033 (~150 Mpc) and distance_kpc < 30
    if not any(
        m.z is not None and m.z <= 0.033 and m.distance_kpc < 30
        for m in cross_matches.ned
    ):
        return False
    return True

In [ ]:
nearby_alerts = []

for a in tqdm(alerts, desc="Filtering nearby alerts"):
    if has_nearby_host(a):
        nearby_alerts.append(a)

print(f"Found {len(nearby_alerts)}/{len(alerts)} nearby alerts.")

In [ ]:
babamul.scan_alerts(nearby_alerts)

In [ ]:
# Now let's demonstrate using the API to look for alerts
survey, object_id = nearby_alerts[0].survey, nearby_alerts[0].objectId
obj = babamul.api.get_object(survey, object_id)
obj.show()

In [ ]:
survey = "ZTF"
alerts_from_api = babamul.api.get_alerts(
    survey,
    start_jd=2460745.5,
    end_jd=2460746.5,
    is_rock=False,
    is_star=False,
    is_near_brightstar=False,
    is_stationary=True,
    min_drb=0.5,
)

print(f"Found {len(alerts_from_api)} alerts matching initial criteria.")

nearby_alerts_from_api = []
for a in tqdm(alerts_from_api, desc="Filtering nearby alerts"):
    if not is_relevant(a):
        continue
    if not has_nearby_host(a):
        continue
    nearby_alerts_from_api.append(a)

print(
    f"Found {len(nearby_alerts_from_api)}/{len(alerts_from_api)} nearby alerts."
)

In [ ]:
babamul.scan_alerts(nearby_alerts_from_api)

In [ ]:
survey = "ZTF"
ra, dec = (
    nearby_alerts_from_api[0].candidate.ra,
    nearby_alerts_from_api[0].candidate.dec,
)
coordinates = SkyCoord(ra=ra, dec=dec, unit="deg", frame="icrs")
radius_arcsec = 2.0
conesearch_results = babamul.api.cone_search_alerts(
    survey,
    coordinates,
    radius_arcsec,
    is_rock=False,
    is_star=False,
    is_near_brightstar=False,
    is_stationary=True,
    min_drb=0.5,
)

for pos_name, alerts in conesearch_results.items():
    print(f"Found {len(alerts)} alerts for position {pos_name}.")
    for a in alerts:
        a.show()

In [ ]:
# let's load the NED LVS catalog at .data/NEDLVS_20250602.fits
ned_lvs: Table = Table.read("data/NEDLVS_20250602.fits")
print(f"Loaded NED LVS catalog with {len(ned_lvs):,} entries.")
# let's keep the nearby galaxies with z <= 0.03 (~150 Mpc)
nearby_galaxies = ned_lvs[ned_lvs["z"] <= 0.03]
print(f"Found {len(nearby_galaxies):,} nearby galaxies at less than 150 Mpc.")

In [ ]:
survey, radius_arcsec = "LSST", 60.0
conesearch_results: dict[str, ZtfAlert | LsstAlert] = (
    babamul.api.cone_search_alerts(
        survey,
        nearby_galaxies,
        radius_arcsec=radius_arcsec,
        is_rock=False,
        is_star=False,
        is_near_brightstar=False,
        is_stationary=True,
        min_drb=0.8,
        n_threads=8,
        batch_size=500,
    )
)

galaxies_with_alerts = [
    name for name, alerts in conesearch_results.items() if alerts
]
print(
    f"Found {len(galaxies_with_alerts)} nearby galaxies with alerts within {radius_arcsec} arcseconds."
)

In [ ]:
hosted_alerts: dict[str, list[ZtfAlert | LsstAlert]] = {}
for alerts in conesearch_results.values():
    for a in alerts:
        a: LsstAlert
        if not a.candidate.isdiffpos:
            continue
        if a.candidate.isDipole:
            continue
        if a.candidate.magpsf > 21.5:
            continue
        if a.candidate.psfFlux_flag:
            continue
        a.get_cross_matches()
        if a.cross_matches.milliquasar:
            continue
        photometry = a.get_photometry()
        # keep those that are < 60 days old
        # and with an amplitude > 1 mag
        first_jd = a.candidate.jd
        faintest_mag, brightest_mag = a.candidate.magpsf, a.candidate.magpsf
        for p in photometry:
            if p.isdiffpos and p.magpsf:
                if p.jd < first_jd:
                    first_jd = p.jd
                if p.magpsf > faintest_mag:
                    faintest_mag = p.magpsf
                if p.magpsf < brightest_mag:
                    brightest_mag = p.magpsf
        age = a.candidate.jd - first_jd if first_jd else None
        amplitude = (
            faintest_mag - brightest_mag
            if faintest_mag and brightest_mag
            else None
        )
        if age is None or age > 60 or age < 10:
            continue
        if amplitude is None or amplitude < 1:
            continue
        # if not is_relevant_complex(a):
        #     continue
        if (
            a.objectId not in hosted_alerts
            or a.candidate.jd > hosted_alerts[a.objectId].candidate.jd
        ):
            hosted_alerts[a.objectId] = a
hosted_alerts = list(hosted_alerts.values())

print(
    f"Found {len(hosted_alerts)} unique hosted alerts around nearby galaxies."
)

In [ ]:
babamul.scan_alerts(hosted_alerts)

In [ ]:
# let's load data/tns_public_objects.csv, which is a dump of all public objects in the Transient Name Server as of 2026-02-12
tns_objects: Table = Table.read("data/tns_public_objects.csv")
print(f"Loaded TNS public objects catalog with {len(tns_objects):,} entries.")

# let's remove those with type AGN
tns_objects = tns_objects[tns_objects["type"] != "AGN"]
# let's remove the already classified supernovae
tns_objects = tns_objects[tns_objects["name_prefix"] != "SN"]
print(
    f"Filtered TNS objects to {len(tns_objects):,} entries after removing AGN."
)

In [ ]:
# do a cone search against the alerts for the tns_sne
survey, radius_arcsec = "LSST", 1.5
conesearch_results = babamul.api.cone_search_alerts(
    survey,
    tns_objects,
    radius_arcsec=radius_arcsec,
    n_threads=8,
    batch_size=500,
)

tns_sne_candidates = [
    name for name, alerts in conesearch_results.items() if alerts
]
print(
    f"Found {len(tns_sne_candidates)} TNS supernovae with alerts within {radius_arcsec} arcseconds:"
)

In [ ]:
# scan for these alerts
alerts_to_scan = {}
for _, alerts in conesearch_results.items():
    for a in alerts:
        if (
            a.objectId not in alerts_to_scan
            or a.candidate.jd > alerts_to_scan[a.objectId].candidate.jd
        ):
            alerts_to_scan[a.objectId] = a
print(f"Found {len(alerts_to_scan)} unique alerts around TNS supernovae.")

babamul.scan_alerts(list(alerts_to_scan.values()))